In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1579226456623_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [ ]:
import time
start = time.time()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession



sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.install_pypi_package('h2o-pysparkling-2.4')
sc.install_pypi_package('h2o')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Created wheel for h2o-pysparkling-2.4: filename=h2o_pysparkling_2.4-3.28.0.1.post1-py2.py3-none-any.whl size=101307807 sha256=0d37adfa93b6dd91b9c6b639495ab8d42514c9ade0bbbca20f3275045128eeb0
  Stored in directory: /var/lib/livy/.cache/pip/wheels/0d/2f/d2/7e0c36c53b57ca597430858ab3d72e1a612a10780016ff6765
  Created wheel for tabulate: filename=tabulate-0.8.6-cp36-none-any.whl size=23271 sha256=5ad4f76001fce55fbc764a3c97aa09611dcc6f92a8793be05bad0f74e82f3398
  Stored in directory: /var/lib/livy/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=247f4559dbadc646e748621d656b3fac36a2abed506a869c2ec804d88c235bf6
  Stored in directory: /var/lib/livy/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=6ae5263c58b19ea62871f0fdd038b7923dc4637db3265798ed8da1c031b

In [4]:
from pysparkling import *
import h2o
h2o.init()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /mnt/tmp/1579226794613-0/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnoqdor26
  JVM stdout: /tmp/tmpnoqdor26/h2o_livy_started_from_python.out
  JVM stderr: /tmp/tmpnoqdor26/h2o_livy_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
--------------------------  ------------------------------------------------------------------
H2O cluster uptime:         01 secs
H2O cluster timezone:       Etc/UCT
H2O data parsing timezone:  UTC
H2O cluster version:        3.28.0.1
H2O cluster version age:    1 month
H2O cluster name:           H2O_from_python_livy_lys6ze
H2O cluster tot

In [5]:
## read from S3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
h2o_LA_left_lag30 = h2o.import_file(
    "s3a://dc-group2/h2o_LA_left_lag30.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Parse progress: [#########################################################] 100%

In [40]:
h2o_LA_left_lag30.set_names(['year', 'month', 'day', 'dow', 'label', 'imputed_wind_speed', 'imputed_temp', 'imputed_pressure', 'imputed_humidity', 'imputed_wind_direction', 'prev1', 'prev2', 'prev3', 'prev4', 'prev5', 'prev6', 'prev7', 'prev8', 'prev9', 'prev10', 'prev11', 'prev12', 'prev13', 'prev14', 'prev15', 'prev16', 'prev17', 'prev18', 'prev19', 'prev20', 'prev21', 'prev22', 'prev23', 'prev24', 'prev25', 'prev26', 'prev27', 'prev28', 'prev29', 'prev30'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  year    month    day    dow    label    imputed_wind_speed    imputed_temp    imputed_pressure    imputed_humidity    imputed_wind_direction    prev1    prev2    prev3    prev4    prev5    prev6    prev7    prev8    prev9    prev10    prev11    prev12    prev13    prev14    prev15    prev16    prev17    prev18    prev19    prev20    prev21    prev22    prev23    prev24    prev25    prev26    prev27    prev28    prev29    prev30
------  -------  -----  -----  -------  --------------------  --------------  ------------------  ------------------  ------------------------  -------  -------  -------  -------  -------  -------  -------  -------  -------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------
  1980        1     31      5      106               3.15382         63.5406             988.592             63.2639                

In [42]:
predictors = h2o_LA_left_lag30.names
response = "label"
predictors.remove(response)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
model_dl = H2ODeepLearningEstimator(variable_importances=True,
                                    nfolds=3,
                                    fold_assignment="Modulo",
                                    keep_cross_validation_predictions=True,
                                    loss="Automatic")

model_dl.train(x=predictors,
               y="label",
               training_frame=h2o_LA_left_lag30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

deeplearning Model Build progress: [######################################] 100%

In [49]:
model_dl

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1579226891109_1


Status of Neuron Layers: predicting label, regression, gaussian distribution, Quadratic loss, 48,401 weights/biases, 585.6 KB, 150,833 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  -------------------  -----------------------
    1        39       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.009081469680367707   0.009372662752866745   0.0         -0.010522196201984628  0.12462973594665527  0.3603261353532649   0.08536380529403687
    3        200      Rectifier  0.0        0.0   0.0   0.07261116620969697    0.076570540

In [ ]:
end = time.time()
gap = end - start
print(gap)